In [66]:
import pandas as pd
import openpyxl
import time

In [67]:
path_file = 'data/СВОД_4 кв_2023.xlsx'

In [68]:
# получаем список листов
temp_wb = openpyxl.load_workbook(path_file,read_only=True)
lst_file = temp_wb.sheetnames
temp_wb.close()

In [69]:
lst_file

['Свод_2019', 'Свод_2020', 'Свод_2021', 'Свод_2022', 'свод_2023']

In [70]:
temp_reg_df = pd.read_excel(path_file,sheet_name='Свод_2019',nrows=1)

In [71]:
# Получаем регионы на листе
temp_lst_regions = [region for region in temp_reg_df.columns if 'Unnamed' not in region]

In [72]:
temp_lst_regions

['Амурская область',
 'Астраханская область',
 'Волгоградская область',
 'Воронежская область',
 'Иркутская область',
 'Кемеровская область',
 'Московская область',
 'Новгородская область',
 'РСО-Алания',
 'Республика Татарстан',
 'Свердловская область',
 'Тверская область',
 'Томская область',
 'Тюменская область']

In [73]:
temp_df = pd.read_excel(path_file,sheet_name='Свод_2019',skiprows=1)

In [74]:
temp_df.head(1)

,№ показателя,Наименование индикатора/показателя,№ строки,Единица измерения,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!),"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)",Фактическое значение показателя нарастающим итогом с года создания ЦОПП,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .1,...,Фактическое значение показателя нарастающим итогом с года создания ЦОПП .10,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).11,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).11",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .11,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).12,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).12",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .12,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).13",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .13
0,1,2,3.0,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# удаляем лишние колонки
temp_df.drop(columns=['№ показателя','№ строки','Единица измерения'],inplace=True)

In [76]:
temp_df.head(5)

,Наименование индикатора/показателя,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!),"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)",Фактическое значение показателя нарастающим итогом с года создания ЦОПП,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .1,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).2,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).2",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .2,...,Фактическое значение показателя нарастающим итогом с года создания ЦОПП .10,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).11,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).11",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .11,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).12,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).12",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .12,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).13",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .13
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Созданы цифровая платформа и сайт ЦОПП в сети ...,NaN,NaN,NaN,https://copp.edu.astrobl.ru/,https://copp.edu.astrobl.ru/,https://copp.edu.astrobl.ru/,NaN,NaN,NaN,...,https://copp66.ru/cifrovaya_platforma\nhttps:/...,создана в 2019 г.,https://copp69.ru/,https://copp69.ru/,https://copp70.ru/,https://copp70.ru/,https://copp70.ru/,https://platform.copp72.ru/ https://www.copp72...,https://platform.copp72.ru/ https://www.copp72...,https://platform.copp72.ru/ https://www.copp72...
3,"Количество обращений, зарегистрированных на ци...",NaN,NaN,NaN,57,252,309,NaN,NaN,NaN,...,9423,679,3860,5477,3633,9167,41027,2870,6003,18693
4,"Численность граждан Российской Федерации, охва...",NaN,NaN,NaN,8651,20046,69707,NaN,NaN,NaN,...,200034,10012,20560,53000,17196,24061,83437,32677,96523,243567


In [83]:
# убираем лишние строки с значением 1 или 2
temp_df = temp_df[~temp_df['Наименование индикатора/показателя'].isin([1,2])]

In [84]:
temp_df.head()

,Наименование индикатора/показателя,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).2","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).3","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).4","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).5","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).6","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).7","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).8","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).9","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).10","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).11","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).12","Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).13"
2,Созданы цифровая платформа и сайт ЦОПП в сети ...,NaN,https://copp.edu.astrobl.ru/,NaN,copp36.ru,"https://cp.copp38.ru , https://copp38.ru /",https://copp42.ru/ https://platform.copp42.ru/,NaN,NaN,NaN,https://copp16.ru https://prof.market,https://copp66.ru/cifrovaya_platforma\nhttps:/...,https://copp69.ru/,https://copp70.ru/,https://platform.copp72.ru/ https://www.copp72...
3,"Количество обращений, зарегистрированных на ци...",NaN,252,NaN,1454,6 на ЦП ЦОПП ИО (остальные обращения идут на о...,15151,NaN,NaN,NaN,251,3105,3860,9167,6003
4,"Численность граждан Российской Федерации, охва...",NaN,20046,NaN,28276,16606,58175,NaN,NaN,NaN,77671,60843,20560,24061,96523
5,из них (из п.3) в возрасте:\n– до 16 лет,NaN,13943,NaN,17370,5906,34774,NaN,NaN,NaN,65851,28093,6237,1377,68121
6,– от 16 до 21 года,NaN,3798,NaN,7442,7813,10657,NaN,NaN,NaN,5396,24047,11264,17455,13164


In [90]:
temp_reg_df = temp_df.copy() # получаем копию датафрейма чтобы получить регионы

In [91]:
# оставляем только колонки за 2023 т.е те что посередине
lst_2023_columns = [column
                    for column in temp_reg_df.columns
                    if 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)'
                   in column
                 ] 

In [92]:
lst_2023_columns

['Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).2',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).3',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).4',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).5',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).6',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).7',
 'Фактическое значение по итогам четырёх кварталов 2023 г. (указыв

In [93]:
# оставляем только колонки с 2023
temp_reg_df = temp_reg_df[lst_2023_columns]

In [94]:
# заменяем названия колонок на регионы
temp_reg_df.columns = temp_lst_regions

In [96]:
temp_reg_df.insert(0,'Наименование индикатора/показателя',temp_df['Наименование индикатора/показателя'])

In [97]:
temp_reg_df

,Наименование индикатора/показателя,Амурская область,Астраханская область,Волгоградская область,Воронежская область,Иркутская область,Кемеровская область,Московская область,Новгородская область,РСО-Алания,Республика Татарстан,Свердловская область,Тверская область,Томская область,Тюменская область
2,Созданы цифровая платформа и сайт ЦОПП в сети ...,NaN,https://copp.edu.astrobl.ru/,NaN,copp36.ru,"https://cp.copp38.ru , https://copp38.ru /",https://copp42.ru/ https://platform.copp42.ru/,NaN,NaN,NaN,https://copp16.ru https://prof.market,https://copp66.ru/cifrovaya_platforma\nhttps:/...,https://copp69.ru/,https://copp70.ru/,https://platform.copp72.ru/ https://www.copp72...
3,"Количество обращений, зарегистрированных на ци...",NaN,252,NaN,1454,6 на ЦП ЦОПП ИО (остальные обращения идут на о...,15151,NaN,NaN,NaN,251,3105,3860,9167,6003
4,"Численность граждан Российской Федерации, охва...",NaN,20046,NaN,28276,16606,58175,NaN,NaN,NaN,77671,60843,20560,24061,96523
5,из них (из п.3) в возрасте:\n– до 16 лет,NaN,13943,NaN,17370,5906,34774,NaN,NaN,NaN,65851,28093,6237,1377,68121
6,– от 16 до 21 года,NaN,3798,NaN,7442,7813,10657,NaN,NaN,NaN,5396,24047,11264,17455,13164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Наличие брендовой сувенирной продукции ЦОПП (р...,NaN,Х,NaN,Х,X,Х,NaN,NaN,NaN,Х,Х,Х,Х,Х
80,Наличие брендовой сувенирной продукции ЦОПП (д...,NaN,Х,NaN,Х,Х,Х,NaN,NaN,NaN,Х,Х,Х,Х,Х
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# очищаем от пустых строк
temp_reg_df.dropna(how=='all')

TypeError: Invalid comparison between dtype=int64 and str